In [ ]:
!pip install datasets transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q wandb
import wandb
!wandb login --relogin "23f2a348ebd46a20e621f4a288a9d744e4dbb6f6"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset

ds = load_dataset('ayang903/jordanclassifier')
ds

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 344
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 52
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 102
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
import os
def transform(example_batch):
    # Take a list of images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # include the labels
    inputs['label'] = example_batch['label']
    return inputs

prepared_ds = ds.with_transform(transform)
os.environ["WANDB_PROJECT"]="train"

# save trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"


In [ ]:
import torch
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments
from transformers import Trainer

metric = load_metric("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

from transformers import ViTForImageClassification

labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  report_to="wandb",
  fp16=False,
  save_steps=20,
  eval_steps=20,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=4,
  remove_unused_columns=False,
  push_to_hub=False,
  load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

<ipython-input-6-8d440809b9e5>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from t

Step,Training Loss,Validation Loss,Accuracy
20,0.843300,0.505466,0.980392
40,0.222200,0.188203,0.960784


Step,Training Loss,Validation Loss,Accuracy
20,0.843300,0.505466,0.980392
40,0.222200,0.188203,0.960784
60,0.079700,0.155509,0.960784
80,0.055400,0.144238,0.960784


***** train metrics *****
  epoch                    =        4.0
  total_flos               = 99307754GF
  train_loss               =     0.3503
  train_runtime            = 0:51:07.28
  train_samples_per_second =      0.449
  train_steps_per_second   =      0.029


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path="/content/vit-base-beans/checkpoint-80",
    path_in_repo = ".",
    repo_id="ayang903/ds340",
    repo_type="model",
    create_pr=1
)

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/687M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/627 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

'https://huggingface.co/ayang903/ds340/tree/refs%2Fpr%2F1/.'